# Individual Assignment WIE3007
Leveraging Featuretools to perform automated feature engineering on datasets.
##### Lee Xiao Yu U2005405

In [2]:
import featuretools as ft
import pandas as pd


### Define entity set and add entities into entity set.

In [3]:
es = ft.EntitySet(id="entity_set")

In [4]:
customers_df = pd.DataFrame({
    'CustomerID': [101, 102, 103],
    'Name': ['John Doe', 'Jane Smith', 'Mike Jordan'],
    'Email': ['john.doe@example.com', 'jane.smith@example.com', 'mike.jordan@example.com'],
    'SignupDate': ['2023-01-10', '2023-01-15', '2023-01-20'],
})
es = es.add_dataframe(dataframe_name="Customers", dataframe=customers_df, index="CustomerID")


In [5]:
products_df = pd.DataFrame({
    'ProductID': [201, 202, 203],
    'Name': ['Laptop', 'Tablet', 'Smartphone'],
    'Category': ['Electronics', 'Electronics', 'Electronics'],
    'Price': [1000, 500, 800],
})
es = es.add_dataframe(dataframe_name="Products", dataframe=products_df, index="ProductID")


In [6]:
orders_df = pd.DataFrame({
    'OrderID': [301, 302, 303],
    'CustomerID': [101, 102, 103],
    'OrderDate': ['2023-02-01', '2023-02-05', '2023-02-10'],
    'ShipDate': ['2023-02-03', '2023-02-07', '2023-02-12'],
})
es = es.add_dataframe(dataframe_name="Orders", dataframe=orders_df, index="OrderID")


In [7]:
order_details_df = pd.DataFrame({
    'OrderID': [301, 302, 303],
    'ProductID': [201, 202, 203],
    'Quantity': [1, 2, 1],
    'Discount': [0, 0.1, 0]
})
es = es.add_dataframe(dataframe_name="OrderDetails", dataframe=order_details_df, make_index=True, index="OrderDetailsID")


### Add relationships to the entities.

In [8]:
relationships = [
    ('Customers', 'CustomerID', 'Orders', 'CustomerID'),
    ('Products', 'ProductID', 'OrderDetails', 'ProductID'),
    ('Orders', 'OrderID', 'OrderDetails', 'OrderID'),  
]

es = es.add_relationships(relationships)

### Entity Set Information

In [9]:
es

Entityset: entity_set
  DataFrames:
    Customers [Rows: 3, Columns: 4]
    Products [Rows: 3, Columns: 4]
    Orders [Rows: 3, Columns: 4]
    OrderDetails [Rows: 3, Columns: 5]
  Relationships:
    Orders.CustomerID -> Customers.CustomerID
    OrderDetails.ProductID -> Products.ProductID
    OrderDetails.OrderID -> Orders.OrderID

### Perform Deep Feature Synthesis to generate features.

In [10]:
# Run deep feature synthesis to generate features
features, feature_names = ft.dfs(entityset=es,
                                 target_dataframe_name='Orders',
                                 verbose=True)


Built 46 features
Elapsed: 00:00 | Progress: 100%|██████████


In [11]:
features

,CustomerID,COUNT(OrderDetails),MAX(OrderDetails.Discount),MAX(OrderDetails.Quantity),MEAN(OrderDetails.Discount),MEAN(OrderDetails.Quantity),MIN(OrderDetails.Discount),MIN(OrderDetails.Quantity),SKEW(OrderDetails.Discount),SKEW(OrderDetails.Quantity),...,Customers.SKEW(OrderDetails.Discount),Customers.SKEW(OrderDetails.Quantity),Customers.STD(OrderDetails.Discount),Customers.STD(OrderDetails.Quantity),Customers.SUM(OrderDetails.Discount),Customers.SUM(OrderDetails.Quantity),Customers.DAY(SignupDate),Customers.MONTH(SignupDate),Customers.WEEKDAY(SignupDate),Customers.YEAR(SignupDate)
OrderID,,,,,,,,,,,,,,,,,,,,,
301,101,1,0.0,1.0,0.0,1.0,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,1.0,10,1,1,2023
302,102,1,0.1,2.0,0.1,2.0,0.1,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.1,2.0,15,1,6,2023
303,103,1,0.0,1.0,0.0,1.0,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,1.0,20,1,4,2023


In [12]:
feature_names

[<Feature: CustomerID>,
 <Feature: COUNT(OrderDetails)>,
 <Feature: MAX(OrderDetails.Discount)>,
 <Feature: MAX(OrderDetails.Quantity)>,
 <Feature: MEAN(OrderDetails.Discount)>,
 <Feature: MEAN(OrderDetails.Quantity)>,
 <Feature: MIN(OrderDetails.Discount)>,
 <Feature: MIN(OrderDetails.Quantity)>,
 <Feature: SKEW(OrderDetails.Discount)>,
 <Feature: SKEW(OrderDetails.Quantity)>,
 <Feature: STD(OrderDetails.Discount)>,
 <Feature: STD(OrderDetails.Quantity)>,
 <Feature: SUM(OrderDetails.Discount)>,
 <Feature: SUM(OrderDetails.Quantity)>,
 <Feature: DAY(OrderDate)>,
 <Feature: DAY(ShipDate)>,
 <Feature: MONTH(OrderDate)>,
 <Feature: MONTH(ShipDate)>,
 <Feature: WEEKDAY(OrderDate)>,
 <Feature: WEEKDAY(ShipDate)>,
 <Feature: YEAR(OrderDate)>,
 <Feature: YEAR(ShipDate)>,
 <Feature: MAX(OrderDetails.Products.Price)>,
 <Feature: MEAN(OrderDetails.Products.Price)>,
 <Feature: MIN(OrderDetails.Products.Price)>,
 <Feature: SKEW(OrderDetails.Products.Price)>,
 <Feature: STD(OrderDetails.Products.Pr

### Generate features of SUM statistics and slect MONTH as transform primitives only.

In [13]:
features_sum, feature_names_sum = ft.dfs(entityset=es,
                                 target_dataframe_name='Orders',
                                 agg_primitives=['SUM'],
                                 trans_primitives=['MONTH'],
                                 verbose=True)


Built 9 features
Elapsed: 00:00 | Progress: 100%|██████████


In [14]:
feature_names_sum

[<Feature: CustomerID>,
 <Feature: SUM(OrderDetails.Discount)>,
 <Feature: SUM(OrderDetails.Quantity)>,
 <Feature: MONTH(OrderDate)>,
 <Feature: MONTH(ShipDate)>,
 <Feature: SUM(OrderDetails.Products.Price)>,
 <Feature: Customers.SUM(OrderDetails.Discount)>,
 <Feature: Customers.SUM(OrderDetails.Quantity)>,
 <Feature: Customers.MONTH(SignupDate)>]

### Explore the features matrix and select useful features for insights
Example: Select product price and quantity for calculation of Total Price. Analyse Total Price with Weekdays to find buying trends.

In [15]:
df_trend = features[['WEEKDAY(OrderDate)','SUM(OrderDetails.Products.Price)', 'SUM(OrderDetails.Quantity)']]
df_trend['Total Price'] = df_trend['SUM(OrderDetails.Products.Price)'] * df_trend['SUM(OrderDetails.Quantity)']
df_trend

,WEEKDAY(OrderDate),SUM(OrderDetails.Products.Price),SUM(OrderDetails.Quantity),Total Price
OrderID,,,,
301,2,1000.0,1.0,1000.0
302,6,500.0,2.0,1000.0
303,4,800.0,1.0,800.0
